In [5]:
import pandas as pd
import warnings

from clean_df_paper import df_training_set
from arma.arma import arma_model

warnings.filterwarnings("ignore")

In [2]:
df_training_set

,SPY US Equity,DIA US Equity,QQQ US Equity
2011-01-03,0.010338,0.006746,0.015608
2011-01-04,-0.000551,0.001976,-0.000814
2011-01-05,0.005198,0.003429,0.008595
2011-01-06,-0.001959,-0.002221,0.003229
2011-01-07,-0.001962,-0.001798,-0.000894
...,...,...,...
2012-12-24,-0.003081,-0.006222,-0.001994
2012-12-26,-0.004215,-0.001222,-0.008760
2012-12-27,-0.001329,-0.001605,-0.001550
2012-12-28,-0.010819,-0.011715,-0.009627


In [7]:
spy_arma = arma_model(df_training_set['SPY US Equity'], (1,1))
dia_arma = arma_model(df_training_set['DIA US Equity'], (1,1))
qqq_arma = arma_model(df_training_set['QQQ US Equity'], (1,1))

In [16]:
arma